In [9]:
import pandas as pd 
data_17_18 = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/DatasetsOriginais/transfermarkt/transfermarkt_fbref_201718.csv', delimiter = ';',  index_col=0)
data_18_19 = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/DatasetsOriginais/transfermarkt/transfermarkt_fbref_201819.csv', delimiter = ';',  index_col=0,  low_memory=False)
data_19_20 = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/DatasetsOriginais/transfermarkt/transfermarkt_fbref_201920.csv', delimiter = ';',  index_col=0)
equipas = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/bundesliga1718.csv', delimiter=',', index_col=0)


In [10]:
# Concatenate the DataFrames vertically

data_17_18['season_year'] = 2018
data_18_19['season_year'] = 2019
data_19_20['season_year'] = 2020

data = pd.concat([data_17_18, data_18_19, data_19_20], ignore_index=True)



In [11]:



# Fazer o merge usando as colunas "squad" e "season_year"
data = data.merge(equipas, left_on=['squad', 'season_year'], right_on=['equipa', 'epoca'], how='left')

print(data)

              player nationality position        squad   age  birth_year  \
0             Burgui      es ESP    FW,MF       Alavés  23.0      1993.0   
1     Raphaël Varane      fr FRA       DF  Real Madrid  24.0      1993.0   
2       Rubén Duarte      es ESP       DF       Alavés  21.0      1995.0   
3      Samuel Umtiti      fr FRA       DF    Barcelona  23.0      1993.0   
4        Manu García      es ESP       MF       Alavés  31.0      1986.0   
...              ...         ...      ...          ...   ...         ...   
7103  Daniel Podence      pt POR    FW,MF       Wolves  23.0      1995.0   
7104   Romain SaĂŻss      ma MAR       DF       Wolves  29.0      1990.0   
7105   Adama TraorĂ©      es ESP    FW,MF       Wolves  23.0      1996.0   
7106  JesĂşs Vallejo      es ESP       DF       Wolves  22.0      1997.0   
7107  RĂşben Vinagre      pt POR       DF       Wolves  20.0      1999.0   

           value  height                      position2   foot  ... xGDiff/90  \
0     

In [12]:
data = data.drop('Season', axis=1)
data = data.drop('Attendance', axis=1)
data = data.drop('equipa', axis =1)
data = data.drop('liga', axis =1)
data = data.drop('squad', axis=1)
data = data.drop('season_year', axis=1)




# Extract country code from the nationality column
data['nationality'] = data['nationality'].str[-3:]

data = data.dropna()

null_counts = data.isnull().sum()
print(null_counts)

data.shape[0]

player           0
nationality      0
position         0
age              0
birth_year       0
                ..
xGDiff/90        0
CL               0
WinCL            0
CLBestScorer     0
classificacao    0
Length: 397, dtype: int64


3579

In [13]:
# Definir o número de bins desejado
num_bins = 5

# Realizar equal-height binning na coluna 'value'
data['bin'] = pd.qcut(data['value'], num_bins, labels=False)

# Obter a contagem de valores em cada bin
bin_counts = data['bin'].value_counts().sort_index()

# Obter os intervalos de valores dos bins
bin_intervals = pd.qcut(data['value'], num_bins, retbins=True)[1]

# Exibir os intervalos de valores dos bins
for i in range(num_bins):
    print(f'Bin {i+1}: {bin_intervals[i]} - {bin_intervals[i+1]}')

# Obter as colunas do DataFrame
colunas = data.columns

# Exibir as colunas
for coluna in colunas:
    print(coluna)

Bin 1: 100.0 - 1000000.0
Bin 2: 1000000.0 - 3000000.0
Bin 3: 3000000.0 - 7000000.0
Bin 4: 7000000.0 - 15000000.0
Bin 5: 15000000.0 - 180000000.0
player
nationality
position
age
birth_year
value
height
position2
foot
league
games
games_starts
minutes
goals
assists
pens_made
pens_att
cards_yellow
cards_red
goals_per90
assists_per90
goals_assists_per90
goals_pens_per90
goals_assists_pens_per90
xg
npxg
xa
xg_per90
xa_per90
xg_xa_per90
npxg_per90
npxg_xa_per90
minutes_90s
shots_total
shots_on_target
shots_free_kicks
shots_on_target_pct
shots_total_per90
shots_on_target_per90
goals_per_shot
goals_per_shot_on_target
npxg_per_shot
xg_net
npxg_net
passes_completed
passes
passes_pct
passes_total_distance
passes_progressive_distance
passes_completed_short
passes_short
passes_pct_short
passes_completed_medium
passes_medium
passes_pct_medium
passes_completed_long
passes_long
passes_pct_long
xa_net
assisted_shots
passes_into_final_third
passes_into_penalty_area
crosses_into_penalty_area
progressive_

In [14]:
# correlação

# Selecione apenas as colunas numéricas

colunas_numericas = data.select_dtypes(include=[float, int])

# Calcule a correlação entre as variáveis numéricas e a variável alvo
correlacao = colunas_numericas.corrwith(data['value'])

# Criar uma lista para armazenar as colunas a serem removidas
colunas_removidas = []

for coluna in colunas_numericas.columns:
    correlacao_valor = correlacao[coluna]
    #print(f"Coluna: {coluna}, Correlação: {correlacao_valor}")
    
    # Verificar se a correlação é negativa
    if correlacao_valor < 0.2:
        colunas_removidas.append(coluna)

# Remover as colunas com correlação negativa do DataFrame
data = data.drop(colunas_removidas, axis=1)

for coluna in colunas_numericas.columns:
    correlacao_valor = correlacao[coluna]
    print(f"Coluna: {coluna}, Correlação: {correlacao_valor}")

data.to_csv("merged_class.csv", index = False)

Coluna: age, Correlação: -0.1117010666781042
Coluna: birth_year, Correlação: 0.027080920834476066
Coluna: value, Correlação: 0.9999999999999998
Coluna: height, Correlação: -0.0012585820552528631
Coluna: games, Correlação: 0.3442349124718368
Coluna: games_starts, Correlação: 0.35083449881199397
Coluna: minutes, Correlação: 0.35116265842915106
Coluna: goals, Correlação: 0.5347473868161405
Coluna: assists, Correlação: 0.5025334162260598
Coluna: pens_made, Correlação: 0.2219101695889232
Coluna: pens_att, Correlação: 0.23566508097809294
Coluna: cards_yellow, Correlação: 0.12079850772742612
Coluna: cards_red, Correlação: 0.030920420776763526
Coluna: goals_per90, Correlação: 0.22800464365018994
Coluna: assists_per90, Correlação: 0.26384478206305034
Coluna: goals_assists_per90, Correlação: 0.29858469440165536
Coluna: goals_pens_per90, Correlação: 0.21962024769426125
Coluna: goals_assists_pens_per90, Correlação: 0.2939858293300613
Coluna: xg, Correlação: 0.5051495851312691
Coluna: npxg, Correla